## Ziyaad Ballim

## Research: Traffic Monitoring with Low-Cost Cameras

In [19]:
!pip install cvlib

In [20]:
import numpy as np
import cv2
import time
import imageio
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
from skimage import color, io
from multiprocessing.connection import wait
from cv2 import erode
import pandas as pd
# import cvlib 
# from cvlib.object_detection import draw_bbox


# import sys 
# import os
# sys.path.append(os.path.abspath("/Users/ziyaadballim/Downloads/Wits/2022/Semester 2/Research/Process.py"))
# from PFrame import ProcessFrames

In [21]:
def VideoMeta(vid):
    frames= int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Frame count: " +str(frames)+" frames" )
    fps    = vid.get(cv2.CAP_PROP_FPS)
    print("Frames Per Second: "+str(fps)+" FPS")
    duration = frames/fps
    print("Video Duration: "+str(duration)+" seconds")
    width  = vid.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    print("Frame Width: "+str(width)+" pixels")
    height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print("Frame Height: "+str(width)+" pixels")

In [22]:
BSMOG= cv2.bgsegm.createBackgroundSubtractorMOG()
BSMOG2 = cv2.createBackgroundSubtractorMOG2()
BSKNN= cv2.createBackgroundSubtractorKNN()
BSGMG= cv2.bgsegm.createBackgroundSubtractorGMG()
BSGSOC= cv2.bgsegm.createBackgroundSubtractorGSOC()

BGSMethods={
"BSMOG":BSMOG,
"BSMOG2":BSMOG2,
"BSKNN":BSKNN,
"BSGMG":BSGMG,
"BSGSOC":BSGSOC
}



classifier = cv2.CascadeClassifier('cars.xml')

In [23]:
    # threshInv = cv2.adaptiveThreshold(gray_frame, 255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # (T, threshInv) = cv2.threshold(gray_frame, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)


    # (T, threshInv) = cv2.threshold(gray_frame, 150, 255,cv2.THRESH_BINARY_INV)

    # th = cv2.adaptiveThreshold(gray_frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1)
    # retval2,threshold2 = cv2.threshold(gray_frame,125,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # retval, threshold = cv2.threshold(gray_frame, 10, 255, cv2.THRESH_BINARY)
    # cv2.imshow('original',img)
    # cv2.imshow('Adaptive threshold',th)

    

    # fgmask = BSMOG2.apply(frame)

    # car_detections = classifier.detectMultiScale(fgmask, 1.1, 1)
    # print(len(car_detections))
    # for (x,y,w,h) in car_detections:
    #     cv2.rectangle(fgmask,(x,y),(x+w,y+h),(255,255,255),2)

    
    # plt.imshow(gray_frame,cmap='gray')
    # plt.show()
    # kernel = np.ones((2, 2), np.uint8)

    # fgmask = cv2.erode(fgmask, kernel, iterations=1)
    # cv2.imshow('fgmask', fgmask)
    # plt.imshow(fgmask,cmap='gray')
    # plt.imshow(fgmask,cmap='gray')
    # plt.show()


In [24]:
def Edge(frame):

    # Setting parameter values
    t_lower = 50  # Lower Threshold
    t_upper = 150  # Upper threshold
    edge = cv2.Canny(frame, t_lower, t_upper)
    cv2.imshow('edge',edge)

In [25]:
def Contours(frame, original):
    (cnt, hierarchy) = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.drawContours(rgb, cnt, -1, (0, 255, 0), 2)
    cv2.drawContours(original, cnt, -1, (0, 255, 0), 2)


    cv2.imshow('contour',rgb)
    # print(len(cnt))
    return len(cnt)

In [58]:
def ProcessFrame(frame,method,ekernel,dkernel):
    gray_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # for i in BGSMethods:
    bs=BGSMethods[method].apply(gray_frame)
    car_detections = classifier.detectMultiScale(bs, 1.1, 1)


    # print(str(method)+": "+str(len(car_detections)))
    # for (x,y,w,h) in car_detections:
    #     cv2.rectangle(bs,(x,y),(x+w,y+h),(0,0,255),2)
    
        # plt.title(i)
        # plt.imshow(bs)
        # plt.show()
    # kernel = np.ones((4, 4), np.uint8)

    eroded=cv2.erode(bs,ekernel,iterations=1)

    # kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(eroded, dkernel, iterations=4)

    # opening=cv2.morphologyEx(bs,cv2.MORPH_OPEN,kernel)
    # closing=cv2.morphologyEx(opening,cv2.MORPH_CLOSE,kernel)
    # closed=cv2.MORPH_CLOSE

    firstrow=np.concatenate((bs,eroded,dilated),axis=1)
    cv2.imshow(method+", eroded, dilated",firstrow)
    # cv2.imshow(method,bs)
    # cv2.imshow('eroded',eroded)
    # cv2.imshow('dilated',dilated)

    carcounts=Contours(dilated,gray_frame)
    # secondrow=np.concatenate((opening,closing),axis=1)
    # cv2.imshow("Opening, Closing",secondrow)
    # cv2.imshow("opening",opening)
    # cv2.imshow("closing",closing)
    # originals=np.concatenate((gray_frame,frame))
    cv2.imshow("footage",frame)
    cv2.imshow("labeled",gray_frame)
    # Contours(gray_frame)
    # time.sleep(1)
    return carcounts


In [59]:
def label(frame):
    box, label, count = cv.detect_common_objects(frame)
    output = draw_bbox(frame, box, label, count)
    cv2.imshow(output)

In [60]:
def AnalyseVideo(video_feed, method,ekernel,dkernel):
    print("Start Analysis")
    # Check if camera opened successfully
    if (video_feed.isOpened()== False): 
        print("Error opening video stream or file")

     #Getting Meta Data about video   
    VideoMeta(video_feed)
    # time.sleep(10)
    # Read until video is completed
    car_counter=np.array(())
    count=0
    while(video_feed.isOpened()):
        # Capture frame-by-frame
        ret, frame = video_feed.read()
        # print(type(frame))
        if ret == True:
            # Display the resulting frame
            # cv2.imshow('Frame',frame)

            # if(count%5==0):
            #     cv2.imwrite('traffic_labels/'+str(count)+".jpg", frame)
            # count+=1
            
            carcounts=ProcessFrame(frame,method,ekernel,dkernel)
            # label(frame)
            if(count%5==0):
                car_counter=np.append(car_counter,carcounts)
            count+=1

            # Edge(frame)
            # print(type(frame))
            # time.sleep(1)

            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        # Break the loop
        else: 
            break
    # When everything done, release the video capture object
    video_feed.release()

    # Closes all the frames
    cv2.destroyAllWindows()

    return car_counter

Main

In [61]:
labels=pd.read_csv("traffic_labels.csv")

In [62]:
labels['Counts']

0     1
1     2
2     2
3     2
4     3
     ..
58    0
59    0
60    0
61    0
62    0
Name: Counts, Length: 63, dtype: int64

In [63]:
labels_array=labels['Counts'].to_numpy()

In [64]:
labels_array

array([1, 2, 2, 2, 3, 4, 4, 3, 3, 3, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [65]:
from sklearn.metrics import accuracy_score

def getAccuracy(Counted, Actual):
    acc=accuracy_score(Actual,Counted)
    # plt.figure(figsize=(8,6))
    # plt.hist(Counted, len((Counted)), alpha=0.5, label="Counted")
    # plt.hist(Actual, len(Actual), alpha=0.5, label="Actual")
    # plt.show()

    print(Counted-Actual)
    return acc

In [68]:
class result:
    def __init__(self, method, esize, dsize, accuracy, total_counts):
        self.method=method
        self.esize=esize
        self.dsize=dsize
        self.accuracy=accuracy
        self.total_counts=total_counts

In [81]:
results=[]

In [82]:

for e in range(1,16):
    for d in range(1,16):
        ekernel=np.ones((e, e), np.uint8)
        dkernel = np.ones((d, d), np.uint8)
        for i in BGSMethods:
            print("Erosion Kernel: "+str(e)+"x"+str(e))
            print("Dilation Kernel: "+str(d)+"x"+str(d))
            print(i)
            video_feed = cv2.VideoCapture('traffic.mp4')
            counts=AnalyseVideo(video_feed,i,ekernel,dkernel)
            print(counts)
            acc=getAccuracy(counts,labels_array)
            print("Accuracy: " +str(acc))
            # break
            print("Next Method")
            cv2.destroyAllWindows()
            res=result(i,e,d,acc,np.sum(counts))
            results.append(res)
            print("________________________________________________________________________")

Erosion Kernel: 1x1
Dilation Kernel: 1x1
BSMOG
Start Analysis
Frame count: 311 frames
Frames Per Second: 23.976023976023978 FPS
Video Duration: 12.971291666666666 seconds
Frame Width: 426.0 pixels
Frame Height: 426.0 pixels
[110. 103. 136. 121. 125. 161. 145. 121. 110.  90.  67.  76.  73. 131.
 122. 106. 108. 115.  59.  38.  37.  49.  81.  85.  58.  23.  12.  26.
  49.  61.  80.  88.  79.  82.  59.  45.  37.  25.  41.  43.  40.  28.
  38.  66.  87.  88. 114. 104. 100.  97.  81.  66. 168.  85. 113. 112.
 117.  82.  84.  73.  69. 150.  43.]
[109. 101. 134. 119. 122. 157. 141. 118. 107.  87.  65.  75.  72. 130.
 120. 104. 106. 113.  57.  36.  36.  49.  81.  85.  58.  23.  12.  26.
  49.  60.  79.  87.  78.  81.  58.  44.  36.  24.  40.  42.  39.  27.
  37.  65.  86.  88. 114. 104. 100.  97.  80.  65. 167.  84. 112. 111.
 116.  82.  84.  73.  69. 150.  43.]
Accuracy: 0.0
Next Method
________________________________________________________________________
Erosion Kernel: 1x1
Dilation Kernel

In [78]:
for i in results:
    print(i.method+" - "+str(i.esize)+" - "+str(i.dsize)+" - "+str(i.accuracy)+" - "+str(i.total_counts))

BSMOG - 1 - 1 - 0.0 - 5200.0
BSMOG2 - 1 - 1 - 0.0 - 19576.0
BSKNN - 1 - 1 - 0.0 - 14719.0
BSGMG - 1 - 1 - 0.3492063492063492 - 320.0
BSGSOC - 1 - 1 - 0.015873015873015872 - 922.0
BSMOG - 1 - 2 - 0.0 - 2018.0
BSMOG2 - 1 - 2 - 0.0 - 5553.0
BSKNN - 1 - 2 - 0.0 - 3634.0
BSGMG - 1 - 2 - 0.4126984126984127 - 213.0
BSGSOC - 1 - 2 - 0.015873015873015872 - 334.0
BSMOG - 1 - 3 - 0.0 - 1381.0
BSMOG2 - 1 - 3 - 0.0 - 3073.0
BSKNN - 1 - 3 - 0.0 - 2032.0
BSGMG - 1 - 3 - 0.4444444444444444 - 170.0
BSGSOC - 1 - 3 - 0.015873015873015872 - 276.0
BSMOG - 1 - 4 - 0.0 - 1068.0
BSMOG2 - 1 - 4 - 0.0 - 1866.0
BSKNN - 1 - 4 - 0.0 - 1415.0
BSGMG - 1 - 4 - 0.4603174603174603 - 149.0
BSGSOC - 1 - 4 - 0.0 - 269.0
BSMOG - 2 - 1 - 0.031746031746031744 - 1434.0
BSMOG2 - 2 - 1 - 0.047619047619047616 - 2391.0
BSKNN - 2 - 1 - 0.031746031746031744 - 2382.0
BSGMG - 2 - 1 - 0.38095238095238093 - 239.0
BSGSOC - 2 - 1 - 0.0 - 1518.0
BSMOG - 2 - 2 - 0.14285714285714285 - 633.0
BSMOG2 - 2 - 2 - 0.07936507936507936 - 1053.0
BSKN

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


/var/folders/vs/8c22bnm90wl78g2hkjd1pdk40000gn/T/ipykernel_7888/4127570118.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
display(labels)

,Frame,CarCounts
0,0,8
1,5,9
2,10,8
3,15,8
4,20,8
5,25,8
6,30,8
7,35,8
8,40,8
9,45,8


In [ ]:
cap = cv2.VideoCapture('video.avi')
fgbg = cv2.createBackgroundSubtractorMOG2()
 
while(1):
    ret, frame = cap.read()
 
    fgmask = fgbg.apply(frame)
  
    cv2.imshow('fgmask', fgmask)
    cv2.imshow('frame',frame )
 
     
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
     
 
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
import numpy as np
import cv2
  
cap = cv2.VideoCapture('video.avi')
  
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
  
# initializing subtractor 
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
  
while(1):
    ret, frame = cap.read()
  
    # applying on each frame
    fgmask = fgbg.apply(frame)
  
    # fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)     
  
    cv2.imshow('frame', fgmask)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
  
cap.release()
cv2.destroyAllWindows()

In [ ]:
img=io.imread("/Users/ziyaadballim/Downloads/Wits/2022/Semester 2/Research/data/training_images/vid_4_1020.jpg")
plt.imshow(img)
plt.show()
print(type(img))
ProcessFrame(img,"BSMOG2")